<a href="https://colab.research.google.com/github/mauriziobrt/schaeffer-dataset/blob/main/riffusion_generation_schaeffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generation

In [ ]:
#@title Install dependencies
!pip install diffusers accelerate

In [ ]:
#@title Install Converter
!git clone https://github.com/mauriziobrt/riffusion_fix_image_audio.git
%pip install argh audiosegment

In [8]:
#@title Move riffusion outside of folder else conversion won't work
!mv "/content/riffusion_fix_image_audio/riffusion/" "/content/riffusion/"

In [9]:
#@title Import libraries
import pydub
import importlib
from riffusion.spectrogram_params import SpectrogramParams
from riffusion.spectrogram_converter import SpectrogramConverter
from riffusion.spectrogram_image_converter import SpectrogramImageConverter

In [10]:
#@title Instantiate converters
param = SpectrogramParams()
spec_conv = SpectrogramConverter(params = param)
spec_img_conv = SpectrogramImageConverter(params= param)
SpectrogramConverter(params= param)

In [ ]:
#@title Load pipeline
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("riffusion/riffusion-model-v1").to("cuda")
pipeline.load_lora_weights("mrzbrt/schaeffer-lora", weight_name="checkpoint-40000/pytorch_lora_weights.safetensors")
#image = pipeline("A pulsar synthesis articulated sound that starts with voice shimmering and ends with a granular synth, distorted").images[0]

Example captions:
- "A pulsar synthesis articulated sound that starts with voice shimmering and ends with a granular synth, distorted"

In [67]:
#@title Parameters for the generation sound objects
images_nr = 10 # @param {type:"integer"}
prompt = "this sound is a flute recording. Only the central part known as body of the instrument is played. The flute is played by blowing inside and mimicking the sound of a trumpet. The fixed note is Sol." # @param {type:"string"}

In [74]:
#@title Load generator
import torch

generator = [torch.Generator(device="cuda").manual_seed(i) for i in range(4)]

In [ ]:
#@title Start Generation
images = pipeline(prompt, generator=generator, num_images_per_prompt=4).images
audios = []
for element in images:
  audios.append(spec_img_conv.audio_from_spectrogram_image(image = element))

  0%|          | 0/50 [00:00<?, ?it/s]

In [73]:
#@title Now choose which index sounds better for you, if they are all bad re-generate new ones.
index = 2 #@param {type: "integer"}
audios[index]

#@markdown Keep the index where you want it, it will be reused fo newer generations!

In [60]:
#@title Fix prompt and decide from which seed to start
prompt = [prompt + t for t in [", articulated", ", complex", ", slow decay", "marked onset"]]
generator = [torch.Generator(device="cuda").manual_seed(index) for i in range(4)]

In [61]:
#@title Generate an example similar to the index you chose
images = pipeline(prompt, generator=generator).images
audios = []
for element in images:
  audios.append(spec_img_conv.audio_from_spectrogram_image(image = element))

  0%|          | 0/50 [00:00<?, ?it/s]

In [66]:
#@title Play the sounds you just generated
index_new = 3 #@param {type: "integer"}
audios[index_new]